In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

from mp3id import mp3ID
from musicBase import musicBase
from musicFinder import musicFinder
from musicPath import pathBasics
from timeUtils import clock, elapsed
from discogsBase import discogs
from multiArtist import multiArtist
from time import sleep
from pandas import DataFrame, Series
from discogsBase import discogs
from ioUtils import getFile
from searchUtils import findDirs
from fileUtils import getDirBasics
from unicodedata import normalize
from fsUtils import moveDir, setDir
from masterdb import getArtistAlbumsDB
## Python Version
import sys
print("Python: {0}".format(sys.version))

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-12-31 20:21:35.584487


In [2]:
disc = discogs()

Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-db/metadata exists
/Volumes/Music/Discog/albums-db/metadata exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists


In [3]:
print("Getting Master Artist DB File")
saveFilename = disc.getMasterSlimArtistDiscogsDBFilename()
print(saveFilename)
discdf = getFile(ifile=saveFilename, debug=False)
artists = list(discdf["Artist"])
print(len(artists))
discdf.head()
# 589630

Getting Master Artist DB File
/Volumes/Music/Discog/db/MasterSlimArtistDB.p
610522


,Name,Artist,Num,DiscArtist
1000,Dave Clarke,Dave Clarke,NaN,Dave Clarke
1000500,Club Pulse,Club Pulse,NaN,Club Pulse
100200,Dike,Dike,NaN,Dike
1002000,Larry Stokes,Larry Stokes,NaN,Larry Stokes
1002600,Gidd Sanchez,Gidd Sanchez,NaN,Gidd Sanchez


In [4]:
artistAlbumsDB = getArtistAlbumsDB(disc)

Current Time is Tue Dec 31, 2019 20:21:38 for 
=================================== Creating Artist Albums DB ===================================
Loading ArtistID Data
Loading data from /Volumes/Music/Discog/db/ArtistIDToAlbumNames.p
  --> This file is 172.6MB.
Loading /Volumes/Music/Discog/db/ArtistIDToAlbumNames.p
Creating Pandas DataFrame for 624924 Artists
	Shape --> (624924, 1)
DataFrame Shape is (624924, 1)
Current Time is Tue Dec 31, 2019 20:23:16 for Done with 
=================================== Creating Artist Albums DB ===================================
Process [{0}] took 1.6 minutes.


In [5]:
def getMusicData(key, artist):
    retval = discdf[discdf[key] == artist]
    if retval.shape[0] > 0:
        return retval
    else:
        return None

In [6]:
artistIDToName = discdf["DiscArtist"].to_dict()
artistNameToID = {}
print("Found {0} ID -> Name entries".format(len(artistIDToName)))
for artistID,artistName in artistIDToName.items():
    if artistNameToID.get(artistName) is None:
        artistNameToID[artistName] = []
    artistNameToID[artistName].append(artistID)
print("Found {0} Name -> ID entries".format(len(artistNameToID)))

Found 610522 ID -> Name entries
Found 565225 Name -> ID entries


In [7]:
#artistDB = disc.getArtistIDToNameData()
#artistNameToID = {v: k for k,v in artistDB.items()}
mulArts  = multiArtist(cutoff=0.9, discdata=artistNameToID, exact=False)

In [8]:
try:
    myMusicNameIDMap = getFile("musicDiscogsMap.p")
    print("Found {0} music <-> discogs maps".format(len(myMusicNameIDMap)))
except:
    myMusicNameIDMap = {}
    print("Could not load music <-> discogs map")

Found 4603 music <-> discogs maps


In [9]:
manualMatches = {"Waterwheel": "Waterwheel",
 "Wally": "Wally (12)",
 "The Violents": "The Violents",
 "The Pressure": "The Pressure (9)",
 "The Pits": "The Pits (3)",
 "The Marquees": "The Marquees",
 "The Crash": "The Crash (5)",
 "Traffic": "Traffic",
 "Tim Reynolds": "Tim Reynolds",
 "Tech-9": "Tech-9 (2)",
 "Taco": "Taco",
 "Sunchild": "Sunchild (4)",
 "Stackabones": "Stackabones",
 "Silencer": "Silencer (8)",
 "Selena": "Selena",
 "Scarecrow": "Scarecrow (22)",
 "Beyond": "Beyond (13)",
 "Chris Lee": "Chris Lee (18)",
                 "Chuck": "Chuck (23)"}

newManual = {}
newManual["Dave Evans"] = "Dave Evans (14)"
newManual["David Cook"] = "David Cook (2)"
newManual["David Rice"] = "David Rice (4)"
newManual["Dead Ringers"] = "Dead Ringers (3)"
newManual["Dies Irae"] = "Dies Irae (4)"
newManual["Dune"] = "Dune (3)"
newManual["Julie Andrews"] = "Julie Andrews"
newManual["Julie Andrews"] = "Julie Andrews"
newManual["Nero"] = "Nero (7)"
newManual["Nancy Moore"] = "Nancy Moore"
newManual["Orwell"] = "Orwell"
newManual["Outcry"] = "Outcry (12)"
newManual["Passengers"] = "Passengers"
newManual["Paul Brewster"] = "Paul Brewster (3)"

newManual["Bob Gibson"] = "Bob Gibson"
newManual["Chaser"] = "Chaser (19)"
newManual["Eightball"] = "Eightball (6)"
newManual["Endless"] = "Endless (6)"
newManual["Endless Struggle"] = "Endless Struggle"
newManual["Eventide"] = "Eventide (5)"
newManual["Fall Of Because"] = "Fall Of Because"
newManual["Fetish"] = "Fetish (5)"
newManual["The Forgotten"] = "The Forgotten"
newManual["Four Jacks"] = "Four Jacks"
newManual["Funkmaster Flex"] = "Funkmaster Flex"
newManual["Gladys"] = "Gladys (14)"
newManual["Glenn Hughes"] = "Glenn Hughes"
newManual["Greenhouse"] = "Greenhouse (2)"
newManual["Grimace"] = "Grimace (18)"
newManual["Groundzero"] = "Groundzero (3)"
newManual["HALO"] = "HALO (5)"
newManual["The Haters"] = "The Haters"
newManual["Haters"] = "The Haters"
newManual["Heritage"] = "Heritage (29)"
newManual["Highstrung"] = "Highstrung (3)"
newManual["I.D.K."] = "I.D.K. (2)"
newManual["Jason Dietz"] = "Jason Dietz"
newManual["Jeff Kelly"] = "Jeff Kelly"
newManual["Jimmie Rodgers"] = "Jimmie Rodgers"
newManual["Lynx"] = "Lynx (13)"
newManual["Stephen Duffy"] = "Stephen Duffy"
newManual["PainKiller"] = "PainKiller"
newManual["Queensrÿche"] = "Queensrÿche"
newManual["38 Special"] = "38 Special (2)"
newManual["Groundzero"] = "Groundzero (3)"
newManual["You"] = "You (21)"
newManual["The Snowmen"] = "The Snowmen"
newManual["The Scientists"] = "The Scientists (2)"
newManual["The Roots"] = "The Roots"
newManual["The Matrix"] = "The Matrix (2)"
newManual["The Criminals"] = "The Criminals"
newManual["The Beards"] = "The Beards"

myMusicNameIDMap.update(manualMatches) 
myMusicNameIDMap.update(newManual) 

print("Found {0} music <-> discogs maps".format(len(myMusicNameIDMap)))

Found 4603 music <-> discogs maps


# Find Music

In [ ]:
musicNameIDMap = {}
multiMap       = {}
for dirN in findDirs("/Users/tgadfort/matched"):
    print(dirN)
    for dirval in findDirs(dirN):        
        artistName = getDirBasics(dirval)[-1]
        artistName = normalize('NFC', artistName)
        
        if myMusicNameIDMap.get(artistName) is not None:
            continue

        mdata = getMusicData("DiscArtist", artistName)
        if not isinstance(mdata, DataFrame):
            matches = mulArts.getArtistNames(artistName)
            if not all(matches.values()):
                print("{0: <10}{1: <50}{2}".format("?", artistName, matches))
                continue
            else:
                print("{0: <10}{1: <50}{2}".format("MIX", artistName, matches))
                srcDir = dirval
                dstDir = setDir("/Volumes/Music/Multi", artistName)
                moveDir(srcDir, dstDir)                
        else:
            if mdata.shape[0] == 1:
                musicNameIDMap[artistName] = [mdata.index, mdata["Name"].values[0]]
            else:
                print("{0: <10}{1: <50}{2}".format("", artistName, ""))
                albums = [x for x in findDirs(dirval) if getDirBasics(x)[-1] not in ["Random", "Todo"]]                
                if len(albums) == 0:
                    print("\tNo Albums\n")
                    srcDir = dirval
                    dstDir = setDir("/Volumes/Music/Random", artistName)
                    moveDir(srcDir, dstDir)
                else:
                    multiMap[artistName] = {"DB": mdata, "Albums": albums}
                    continue
                    for album in albums:
                        print("\t",album)
                    print("")
                print("\tDiscogs Data:")
                print(mdata)
                print("\n\n")

In [ ]:
newManual = {}
newManual[""] = ""





In [ ]:
print(len(multiMap))
import json
json.dumps(list(multiMap.keys()))

In [ ]:
from difflib import SequenceMatcher

def getRowByIndex(pdf, idx):
    return pdf.loc[idx]

In [ ]:
ratVal = 0.6
if len(myMusicNameIDMap) > 0:
    ratVal = 0.3
artistIDMap = {}
for artistName,artistSlimData in multiMap.items():
    print("\n","="*50)
    print("ArtistName: {0}".format(artistName))
    print("   Albums: {0}".format(len(artistSlimData["Albums"])))

    myAlbumNames = []
    for album in artistSlimData["Albums"]:
        myAlbumName = album.split("/{0}/".format(artistName))[-1]
        #print("     {0: <15}{1: <10}{2}".format("", "", myAlbumName))
        myAlbumNames.append(myAlbumName)
        
    for idx,row in artistSlimData["DB"].iterrows():
        artistAlbumsData = getRowByIndex(artistAlbumsDB, idx)
        artistAlbums     = artistAlbumsData["Albums"]
        print("   Match: {0}  [{1}]".format(row["DiscArtist"], row["Name"]))
        if isinstance(artistAlbums, dict):
            for albumType, albumTypeData in artistAlbums.items():
                for albumID, dbAlbumName in albumTypeData.items():
                    for myAlbumName in myAlbumNames:
                        s = SequenceMatcher(None, myAlbumName, dbAlbumName)
                        ratio = s.ratio()
                        if ratio > 0.6:
                            print("     {0: <15}{1: <10}{2: <8}{3}".format(albumType, albumID, round(ratio,2), dbAlbumName))
                            artistIDMap[artistName] = [idx, row["Name"]]

In [ ]:
for artistName,artistData in artistIDMap.items():
    print(artistName,' ==> ',artistData)

In [ ]:
print(len(artistIDMap))
if len(myMusicNameIDMap) == 0:
    myMusicNameIDMap = {}
    print(len(musicNameIDMap))
    myMusicNameIDMap = {k: [v[0][0], v[1].values[0]] for k,v in musicNameIDMap.items()}
print(len(myMusicNameIDMap))
myMusicNameIDMap.update(artistIDMap)
print(len(myMusicNameIDMap))

In [ ]:
from ioUtils import saveFile
saveFile(ifile="musicDiscogsMap.p", idata=myMusicNameIDMap, debug=True)

In [ ]:
myMusicNameIDMap = getFile("musicDiscogsMap.p")
print("Found {0} music <-> discogs maps".format(len(myMusicNameIDMap)))

In [ ]:
from searchUtils import findW